In [83]:
import json
import fiona
import os
import re
import geocoder
import csv
import rtree
from shapely.geometry import Point, MultiPoint, shape, mapping
import pyproj

In [9]:
os.chdir('../../../data_generation/data/ATRS/')


In [73]:
MAP_FP = '../maps'
PROJ = pyproj.Proj(init='epsg:3857')

In [12]:
atrs = os.listdir(os.getcwd())

In [57]:
atr_split = [x.split('_') for x in atrs]
atr_address = [' '.join(x[3:5]) for x in atr_split]
atr_address = [re.sub('-', ' ', x) for x in atr_address]
# just unique ones
atr_address = set(atr_address)

In [36]:
g = geocoder.google(atr_address[0] + ' Boston, MA')

In [58]:
# geocode, parse result - address, lat long
results = []
for add in atr_address:
    g = geocoder.google(add + ' Boston, MA')
    r = [add, g.address, g.lat, g.lng]
    results.append(r)

In [60]:
with open('geocoded_atrs.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['orig','geocoded', 'lat', 'lng'])
    for r in results:
        writer.writerow(r)

In [80]:
def read_shp(fp):
    """ Read shp, output tuple geometry + property """
    out = [(shape(line['geometry']), line['properties'])
           for line in fiona.open(fp)]
    return(out)
def read_record(record, x, y, orig=None, new=PROJ):
    """
    Reads record, outputs dictionary with point and properties
    Specify orig if reprojecting
    """
    if (orig is not None):
        x, y = pyproj.transform(orig, new, x, y)
    r_dict = {
        'point': Point(float(x), float(y)),
        'properties': r
    }
    return(r_dict)
def find_nearest(records, segments, segments_index, tolerance):
    """ Finds nearest segment to records
    tolerance : max units distance from record point to consider
    """

    print "Using tolerance {}".format(tolerance)

    for record in records:
        record_point = record['point']
        record_buffer_bounds = record_point.buffer(tolerance).bounds
        nearby_segments = segments_index.intersection(record_buffer_bounds)
        segment_id_with_distance = [
            # Get db index and distance to point
            (
                segments[segment_id][1]['id'],
                segments[segment_id][0].distance(record_point)
            )
            for segment_id in nearby_segments
        ]
        # Find nearest segment
        if len(segment_id_with_distance):
            nearest = min(segment_id_with_distance, key=lambda tup: tup[1])
            db_segment_id = nearest[0]
            # Add db_segment_id to record
            record['properties']['near_id'] = db_segment_id
        # If no segment matched, populate key = ''
        else:
            record['properties']['near_id'] = ''

In [67]:
#Read in segments
inter = read_shp(MAP_FP + '/inters_segments.shp')
non_inter = read_shp(MAP_FP + '/non_inters_segments.shp')
print "Read in {} intersection, {} non-intersection segments".format(len(inter), len(non_inter))

Read in 8574 intersection, 17388 non-intersection segments


In [84]:
# Combine inter + non_inter
combined_seg = inter + non_inter

# Create spatial index for quick lookup
segments_index = rtree.index.Index()
for idx, element in enumerate(combined_seg):
    segments_index.insert(idx, element[0].bounds)

In [96]:
# Read in atr lats
atrs = []
with open('geocoded_atrs.csv') as f:
    csv_reader = csv.DictReader(f)
    for r in csv_reader:
        # Some crash 0 / blank coordinates
        if r['lat']!='':
            atrs.append(
                read_record(r, r['lng'], r['lat'],
                           orig=pyproj.Proj(init='epsg:4326'))
            )
print "Read in data from {} atrs".format(len(atrs))

Read in data from 253 atrs


In [97]:
# Find nearest atr - 20 tolerance
print "snapping atr to segments"
find_nearest(atrs, combined_seg, segments_index, 30)

snapping atr to segments
Using tolerance 30


In [102]:
atrs[0]

{'point': <shapely.geometry.point.Point at 0x11e5d9f90>,
 'properties': {'geocoded': '31 Glenville Ave, Boston, MA 02134, USA',
  'lat': '42.3504522',
  'lng': '-71.1331825',
  'near_id': 6481,
  'orig': '31 GLENVILLE AVE'}}

In [103]:
with open('snapped_atrs.json', 'w') as f:
    json.dump([x['properties'] for x in atrs], f)